# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [89]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [90]:
total_schools = school_data_complete["school_name"].count()
total_students = school_data_complete["student_name"].count()
total_budget = school_data_complete["budget"].sum()
average_math_score = school_data_complete["math_score"].mean()
average_reading_score = school_data_complete["reading_score"].mean()
num_passing_math = school_data_complete.loc[school_data_complete["math_score"] > 69].count()["student_name"]
percent_passing_math = (num_passing_math / total_students) * 100
num_passing_reading = school_data_complete.loc[school_data_complete["reading_score"] > 69].count()["student_name"]
percent_passing_reading = (num_passing_reading / total_students) * 100
percent_overall_passing = (average_math_score + average_reading_score) / 2
district_summary = pd.DataFrame({"Total Schools": [total_schools], "Total Students": [total_students], "Total Budget": [total_budget], "Average Math Score": [average_math_score], "Average Reading Score": [average_reading_score], "% Passing Math": [percent_passing_math], "% Passing Reading": [percent_passing_reading], "% Overall Passing Rate": [percent_overall_passing]})
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,39170,39170,82932329558,78.985371,81.87784,74.980853,85.805463,80.431606


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [91]:
grouped_schools = school_data_complete.groupby(["School ID"])

SchoolCount = pd.DataFrame(grouped_schools["school_name"].size())
SchoolCount = SchoolCount.rename(columns={"school_name": "total_students"})

In [92]:
TotalBudget = pd.DataFrame(grouped_schools["budget"].sum())
TotalBudget = TotalBudget.rename(columns={"budget": "total_budget"})

In [93]:
MathAvgdf = pd.DataFrame(grouped_schools["math_score"].mean())
MathAvgdf = MathAvgdf.rename(columns={"math_score": "avg_math_score"})
ReadingAvgdf = pd.DataFrame(grouped_schools["reading_score"].mean())
ReadingAvgdf = ReadingAvgdf.rename(columns={"reading_score": "avg_reading_score"})

In [94]:
dfPassingMath = school_data_complete[school_data_complete["math_score"] > 69]
grouped_passingmath = dfPassingMath.groupby("School ID")
dfPassingMath = grouped_passingmath["student_name"].count().reset_index()
dfPassingMath = dfPassingMath.rename(columns={"student_name": "num_stu_passing_math"})

In [95]:
dfPassingRead = school_data_complete[school_data_complete["reading_score"] > 69]
grouped_passingread = dfPassingRead.groupby("School ID")
dfPassingRead = grouped_passingread["student_name"].count().reset_index()
dfPassingRead = dfPassingRead.rename(columns={"student_name": "num_stu_passing_reading"})

In [96]:
school_summary = pd.merge(SchoolCount, TotalBudget, on="School ID")
school_summary = pd.merge(school_summary, MathAvgdf, on="School ID")
school_summary = pd.merge(school_summary, ReadingAvgdf, on="School ID")
school_summary = pd.merge(school_summary, dfPassingMath, on="School ID")
school_summary = pd.merge(school_summary, dfPassingRead, on="School ID")

In [97]:
budget_per_student = school_summary["total_budget"] / school_summary["total_students"]
school_summary["per_student_budget"] = budget_per_student
percent_pass_math = school_summary["num_stu_passing_math"] / school_summary["total_students"]
school_summary["percent_passing_math"] = percent_pass_math
percent_pass_read = school_summary["num_stu_passing_reading"] / school_summary["total_students"]
school_summary["percent_passing_reading"] = percent_pass_read
overall_passing_rate = (school_summary["percent_passing_math"] + school_summary["percent_passing_reading"]) / 2
school_summary["overall_passing_rate"] = overall_passing_rate

school_summary.head()

,School ID,total_students,total_budget,avg_math_score,avg_reading_score,num_stu_passing_math,num_stu_passing_reading,per_student_budget,percent_passing_math,percent_passing_reading,overall_passing_rate
0,0,2917,5573322295,76.629414,81.182722,1916,2372,1910635.0,0.656839,0.813164,0.735002
1,1,2949,5557128039,76.711767,81.158020,1946,2381,1884411.0,0.659885,0.807392,0.733639
2,2,1761,1860672600,83.359455,83.725724,1653,1688,1056600.0,0.938671,0.958546,0.948609
3,3,4635,14007062700,77.289752,80.934412,3094,3748,3022020.0,0.667530,0.808630,0.738080
4,4,1468,1346890000,83.351499,83.816757,1371,1426,917500.0,0.933924,0.971390,0.952657


In [104]:
school_summary = pd.merge(school_summary, school_data, on="School ID")
school_summary = school_summary.rename(columns={"school_name_x": "School Name", "type_x": "School Type"})
reorg_school_summary = school_summary[["School ID","School Name","School Type","total_students","total_budget","per_student_budget","avg_math_score","avg_reading_score","percent_passing_math","percent_passing_reading","overall_passing_rate"]]
reorg_school_summary.head()

,School ID,School Name,School Name,School Type,School Type,total_students,total_budget,per_student_budget,avg_math_score,avg_reading_score,percent_passing_math,percent_passing_reading,overall_passing_rate
0,0,Huang High School,Huang High School,District,District,2917,5573322295,1910635.0,76.629414,81.182722,0.656839,0.813164,0.735002
1,1,Figueroa High School,Figueroa High School,District,District,2949,5557128039,1884411.0,76.711767,81.158020,0.659885,0.807392,0.733639
2,2,Shelton High School,Shelton High School,Charter,Charter,1761,1860672600,1056600.0,83.359455,83.725724,0.938671,0.958546,0.948609
3,3,Hernandez High School,Hernandez High School,District,District,4635,14007062700,3022020.0,77.289752,80.934412,0.667530,0.808630,0.738080
4,4,Griffin High School,Griffin High School,Charter,Charter,1468,1346890000,917500.0,83.351499,83.816757,0.933924,0.971390,0.952657


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [112]:
reorg_school_summary = reorg_school_summary.sort_values("overall_passing_rate", ascending=False)
reorg_school_summary.head(5)

,School ID,School Name,School Name,School Type,School Type,total_students,total_budget,per_student_budget,avg_math_score,avg_reading_score,percent_passing_math,percent_passing_reading,overall_passing_rate
6,6,Cabrera High School,Cabrera High School,Charter,Charter,1858,2009159448,1081356.0,83.061895,83.975780,0.941335,0.970398,0.955867
14,14,Thomas High School,Thomas High School,Charter,Charter,1635,1705517550,1043130.0,83.418349,83.848930,0.932722,0.973089,0.952905
9,9,Pena High School,Pena High School,Charter,Charter,962,563595396,585858.0,83.839917,84.044699,0.945946,0.959459,0.952703
4,4,Griffin High School,Griffin High School,Charter,Charter,1468,1346890000,917500.0,83.351499,83.816757,0.933924,0.971390,0.952657
5,5,Wilson High School,Wilson High School,Charter,Charter,2283,3012587442,1319574.0,83.274201,83.989488,0.938677,0.965396,0.952037


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [113]:
reorg_school_summary = reorg_school_summary.sort_values("overall_passing_rate", ascending=True)
reorg_school_summary.head(5)

,School ID,School Name,School Name,School Type,School Type,total_students,total_budget,per_student_budget,avg_math_score,avg_reading_score,percent_passing_math,percent_passing_reading,overall_passing_rate
11,11,Rodriguez High School,Rodriguez High School,District,District,3999,10186904637,2547363.0,76.842711,80.744686,0.663666,0.802201,0.732933
1,1,Figueroa High School,Figueroa High School,District,District,2949,5557128039,1884411.0,76.711767,81.158020,0.659885,0.807392,0.733639
0,0,Huang High School,Huang High School,District,District,2917,5573322295,1910635.0,76.629414,81.182722,0.656839,0.813164,0.735002
12,12,Johnson High School,Johnson High School,District,District,4761,14733628650,3094650.0,77.072464,80.966394,0.660576,0.812224,0.736400
13,13,Ford High School,Ford High School,District,District,2739,4831365924,1763916.0,77.102592,80.746258,0.683096,0.792990,0.738043


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [121]:
def average_scores_by_grade(type):
    col = f"{type}_score"

ninth_grade = school_data_complete.loc[school_data_complete["grade"] == "9th"].groupby("school_name").[col].mean().reset_index()
ninth_grade.head()    

SyntaxError: invalid syntax (<ipython-input-121-5ad00da58d44>, line 1)

## Reading Score by Grade 

* Perform the same operations as above for reading scores

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [17]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


## Scores by School Size

* Perform the same operations as above, based on school size.

In [ ]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

* Perform the same operations as above, based on school type.

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
